In [7]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict, load_model, Metrics, save_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# process_train_data()
# process_test_data()

train_data, test_data = load_split_processed_data()

model_params = {
    "random_state": 42,
    "seed": 42,
    "objective": "binary",
    "metric": "average_precision",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "feature_pre_filter": False,
    "lambda_l1": 2.549324596128614,
    "lambda_l2": 5.886365933437645,
    "learning_rate": 0.04281241059661248,
    "num_leaves": 109,
    "feature_fraction": 0.9080048414519606,
    "bagging_fraction": 0.8771663737338224,
    "max_depth": 10,
    "early_stopping_rounds": 126,
}


train_x = train_data.drop(columns="target")
train_y = train_data.target

valid_x = test_data.drop("target", axis=1)[train_x.columns]
valid_y = test_data.target

dtrain = lgb.Dataset(train_x, label=train_y)
dvalid = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)

model = lgb.train(
    model_params,
    dtrain,
    valid_sets=[dvalid],
)

X = test_data[train_x.columns]
y_true = test_data.target

y_pred_proba = model.predict(X, num_iteration=model.best_iteration)
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

Metrics().call(y_true, y_pred, y_pred_proba)

save_model(model, "1.BEST_LightGbmV2_p072_r031_090auc_new_features_after_dropcorrelations", list(train_x.columns))


Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.72      0.31      0.44      9403

    accuracy                           0.95    150000
   macro avg       0.84      0.65      0.71    150000
weighted avg       0.94      0.95      0.94    150000

Save 1.BEST_LightGbmV2_p072_r031_090auc_new_features_after_dropcorrelations


'1.BEST_LightGbmV2_p072_r031_090auc_new_features_after_dropcorrelations'